!pip install torch torchvision torchaudio
!pip install torch_geometric

In [1]:
!pip install torch torchvision torchaudio
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=4d666126e7ae367e4581c84b64a6d3c4bb3e2559b953a8ea938769a4a12ee580
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


from google.colab import drive
drive.mount('/content/drive')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir(r'/content/drive/MyDrive/AS')

In [4]:
import os
import sys
import time
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

import torch
from torch_geometric.loader import DataLoader, DataListLoader
from torch_geometric.data import Batch
# from torch_geometric.nn import DataParallel
import graphClasses as gf
# import edgePruning as ep
import v2_visFunctions as vf

import json
import pickle
import importlib

In [5]:
print(f"Let's use {torch.cuda.device_count()} GPUs!")

Let's use 1 GPUs!


In [6]:
my_seed = 830

np.random.seed(my_seed)
torch.manual_seed(my_seed)
torch.cuda.manual_seed(my_seed)
torch.backends.cudnn.deterministic = True

# Active Learning

Learn models as data is accessed from the pool via the specified query selection method.

The 100 meshes in the MPI dataset is actually comprised of 10 samples each wih 10 poses. To reduce computational cost but to preserve logic given the nature of this dataset, our active learning queries all poses from the patient selected at each round. Specifically, we sum the disagreement from all poses for each patient, and select the patient with highest total disagreement across poses. We query all poses from this patient for the next round of active learning.

In [7]:
def NewOnlineLearning(dim_h, dim_in, model_dir, pred_dir, start_n, end_n, num_instances, querySelection, X, epochs,
                      imgobj,
                      mc_repeats:int=10, 
                      mc_prob:float=0.3,
                      load_model:bool=False):  


    print("="*30, f'Starting {str(querySelection).upper()}','='*30)
    base_model_dir = model_dir.replace(querySelection, 'passive')
    X_L, X_U, train_df, unobs_df = base_run(X, base_model_dir, load_model=load_model)

    log_path = model_dir+"log.txt"


    for instance_i, split_i in enumerate(range(start_n,end_n)):
        
        print(f'\n>>> Querying instance {instance_i+1} with {str(querySelection).upper()}..........')

        model_fname = f'/al_instance{instance_i}'
        model_path = model_dir+model_fname

        if querySelection == 'passive':

            # Passive learning
            xstar_i, model, predict_probs, train_df, unobs_df = passive_learning(dim_h, dim_in, device, X_L, X_U, epochs, train_df, unobs_df)        

            print(f'PASSIVE QUERIED INDEX: {xstar_i}') 


#         elif querySelection == "qbc":
            
            ####################################
#             committee_size = 3
            
#             labeled_list = [pose for patient_poses in X_L for pose in patient_poses]
            
#             X_Ls = Bootstrap(committee_size, labeled_list)
#             print(len(X_Ls), len(X_Ls[0]), type(X_Ls[0][0]))

#             print("Training query committee.....")
#             xstar_i, _, _, consensus_probs = QBC(dim_h, dim_in, device, X_Ls, X_U, instance_i, epochs)

#             print(f'QBC QUERIED INDEX: {xstar_i}')
            # print('QBC consensus_probs type',type(consensus_probs))

            
            # test_list = [pose for patient_poses in X_U for pose in patient_poses]
            # if run_i == 0:
            #     imgobj.create_images(test_list, torch.tensor(consensus_probs), num_instances, num_poses, instance_i, querySelection)
            #     imgobj.track_index(xstar_i,num_poses)
                
            #####################################
            
#             X_L = X_L + [X_U[xstar_i]]
#             X_U = X_U[:xstar_i] + X_U[xstar_i+1:]
#             print(len(X_L), len(X_U))
# #             print(len(X_L[0]), len(X_U[0]))

        elif querySelection == "mc":

            xstar_i, model, predict_probs, train_df, unobs_df = MC_dropout(dim_h, dim_in, device, X_L, X_U, epochs, mc_prob, mc_repeats,
                                                                           train_df, unobs_df)
            print(f'MCD QUERIED INDEX: {xstar_i}')


        elif querySelection == "uns":

            xstar_i, model, predict_probs, train_df, unobs_df = uncertainty_sampling(dim_h, dim_in, device, X_L, X_U, epochs, train_df, unobs_df)
            print(f'UNS QUERIED INDEX: {xstar_i}')

        elif querySelection == "dbs":

            xstar_i, model, predict_probs, train_df, unobs_df = dbs_sampling(dim_h, dim_in, device, X_L, X_U, epochs, train_df, unobs_df)
            print(f'DBS QUERIED INDEX: {xstar_i}')         

        elif querySelection == "qbc":

            xstar_i, model, predict_probs, train_df, unobs_df = QBC(dim_h, dim_in, device, X_L, X_U, instance_i, epochs, train_df, unobs_df)
            print(f'QBC QUERIED INDEX: {xstar_i}')   


        print('X_L, X_U',len(X_L), len(X_U))

        gf.save_model(model, model_path)

        # f = open(os.path.join(pred_dir,f'al_instance{instance_i}.pckl'),'wb')
        # pickle.dump(predict_probs, f)
        # f.close()

        # if run_i == 0:
            # if querySelection is not 'passive':
            #     imgobj.create_images_single(test_list, predict_probs, num_instances, instance_i, querySelection.upper())
            # else:
            #     imgobj.create_images_single(test_list, predict_probs, num_instances, instance_i, 'Passive')
            # imgobj.track_index_single(xstar_i)

        X_L = X_L + [X_U[xstar_i]]
        del X_U[xstar_i]

        with open(log_path, 'a') as f:
            f.write(str(xstar_i) + "\n")   
    
    return train_df.to_numpy(), unobs_df.to_numpy()


# Query By Committee

Committee size is 3.

**Bootstrapping.** Each bootstrapped dataset contains datasets of the same size as the training set at each round. Given a training set, $D$ of size $n$, Bagging involves creating $m$ new training sets of size $n$ by sampling from $D$ uniformly and with replacement (i.e., bootstrapping)

**Quanify Disagreement across committee for each sample.** Our model outputs are log softmax probabilities for class, for each node, for each testing mesh; thus the committee is a tensor with shape [$m$, size of test set, number of nodes, number of classes]. In our case, the shape in the first round of active learning is [5, 50, 6890, 12]. The consensus is the average probability of each class and has shape [50, 6890, 12]. 

In [8]:
def Bootstrap(cmt_size, X_L):
    # print(len(X_L))
    X_Ls = []
    numsamples = len(X_L)
    for _ in range(cmt_size):
        rand_i = np.random.choice(range(len(X_L)),size=numsamples,replace=True)
        
        X_Lbooted = [X_L[get_i] for get_i in rand_i]

        X_Ls.append(X_Lbooted)
    
    return X_Ls

In [9]:
def QuantifyLargestPatientDisagreement(probs_a, consensus, num_poses):
    '''
    Our active learning queries all poses from the patient selected. Pilot uses 3 poses per patient.
    '''
    
    # probs_a dimensions: number of committee members, number of testing meshes, number of nodes, number of classes
    # consensus dimensions: number of testing meshes, number of nodes, number of classes
    
#     disagreements = np.zeros((probs_a.shape[:-1]))
    
    print('Calculating committee disagreement...')
    print(probs_a.shape)
    print(consensus.shape)
    # print(np.sum(probs_a - np.expand_dims(consensus, axis=0)))
    x = np.nan_to_num(probs_a / consensus)
    disagreements = np.log(x) * probs_a
    # disagreements = np.log(probs_a / consensus) * probs_a
    print(disagreements.shape)
    print(type(disagreements))
    dis = np.sum(disagreements, axis=-1)
    print(dis.shape)
    dis2 = np.sum(dis, axis=0)
    print(dis2.shape)
    dis3 = np.sum(dis2, axis=-1)
    print(dis3.shape)
    print(dis3)
    istar = np.argmax(dis3)
    print(istar)
    i_star = istar // num_poses
  

    return int(i_star)

In [10]:
def QuantifyLargestDisagreement(probs_a, consensus, num_poses):
    '''
    Our active learning queries all poses from the patient selected. Pilot uses 3 poses per patient.
    '''
    
    # probs_a dimensions: number of committee members, number of testing meshes, number of nodes, number of classes
    # consensus dimensions: number of testing meshes, number of nodes, number of classes
    
#     disagreements = np.zeros((probs_a.shape[:-1]))
    
    print('Calculating committee disagreement...')
    # print(probs_a.shape)
    # print(consensus.shape)
    # print(np.sum(probs_a - np.expand_dims(consensus, axis=0)))
    x = np.nan_to_num(probs_a / consensus)
    disagreements = np.log(x) * probs_a
    # disagreements = np.log(probs_a / consensus) * probs_a
    # print(disagreements.shape)
    # print(type(disagreements))
    dis = np.sum(disagreements, axis=-1)
    # print(dis.shape)
    dis2 = np.sum(dis, axis=0)
    # print(dis2.shape)
    dis3 = np.sum(dis2, axis=-1)
    # print(dis3.shape)
    # print(dis3)
    istar = np.argmax(dis3)
    # print(istar)
    i_star = istar // num_poses   

    return int(i_star)

In [11]:
def softvote(proba,
             num_cmt:int=3):
	
	"""
	Calculate uncertainty using soft vote entropy and return index of point with highest soft vote entropy for quantifying QBC disagreement
	proba dimensions = (# cmt, # samples, # nodes, # classes)
    """

	probM = np.sum(proba, axis=0)/num_cmt
	probM = probM*np.log(probM)
	sve = -np.sum(np.sum(probM, axis=-1), -1)
	istar = np.argmax(sve)
	
	return istar


In [22]:
def QBC(dim_h, dim_in, device, X_L, X_U, instance_i, epochs, train_df, unobs_df, num_cmt:int=3):
    
    # populate data matrix with dimensions: number of committee members, number of test patients * number of poses per patient, number of nodes, number of classes
    num_test_patient = len(X_U)

    probs_cmt = np.zeros((num_cmt, num_test_patient, X_U[0].num_nodes, 12))
    # print('probs_cmt.shape',probs_cmt.shape)

    test_arr = np.zeros((num_cmt, 4))
    train_arr = np.zeros((num_cmt, 4))

    X_LL = Bootstrap(num_cmt, X_L)
    
    start_time_2 = time.time()
    
    # For each committee, 1) train, 2) test, 3) use log softmax for every pose to quantify disagreement among committee
    for i, x_cmt in enumerate(X_LL):
        print(f'\n>>>>> Entering committe member {i+1}...')
        
        model_fname = f'/al_instance{instance_i}_cmt{i}'
        model_path = model_dir+model_fname
        
        model = gf.GraphSAGE(dim_h, dim_in, 12)
            # model = DataParallel(model)
        model.to(device)
        
        ################################
        #split data in train and validation
        val_train_split = 1
    
        train_list = x_cmt[val_train_split:]
        val_list = x_cmt[:val_train_split]
        test_list = X_U       
    
        model = gf.GraphSAGE(dim_h, dim_in, 12)
#     model = DataParallel(model)
        model.to(device)
            
        # print('Constructing training data.')
        train_data = DataLoader(train_list, batch_size=6, shuffle=True)

        # print('Constructing validation data.')
        val_data = DataLoader(val_list, batch_size=1, shuffle=True)

        print(f'Training model cmt {i}...')
        start_time = time.time()
        train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
        train_arr[i] = train_acc, train_dice, train_iou, train_loss.cpu().numpy()  
        print(f'Total time taken for UNS training: {time.time()-start_time:.4f} seconds')

        # print('Constructing test data.')
        test_data = DataLoader(test_list, batch_size=1, shuffle=False)

        #Repeat test runs
        print('Running test graphs.')
        loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
        test_arr[i] = acc, dice, iou, loss.cpu().numpy()

        print(f'Loss: {loss}, Acc: {acc}, IOU: {iou}, DICE: {dice}')
        print(f'Total time taken for perform UNS predictions: {time.time()-start_time:.4f} seconds')
        
        probs_cmt[i,:,:,:] = predict_probs.cpu()

        # print(f'Testing loss: {loss:.2f}')
        # print(f'Testing accuracy: {acc:.2f}')
        # print('-'*40)
    
        print(f'Total time for training and testing committee memmber {i+1}: {time.time()-start_time_2:.4f} ')

    train_df[len(unobs_df.columns)] = list(np.mean(train_arr, axis=0)) 
    unobs_df[len(unobs_df.columns)] = list(np.mean(test_arr, axis=0)) 

    # xstar_i = QuantifyLargestDisagreement(probs_cmt, consensus, num_patient_poses)
    xstar_i = softvote(probs_cmt)    

    return xstar_i, model, probs_cmt, train_df, unobs_df

# Using MC dropout

**Number of repeats** Number of repeats for test run is 2

**Dropout probability** The probability of dropouts is 0.3

**Point selection** Entropy is used to select the next instance  

In [13]:
def MC_dropout(dim_h, dim_in, device, X_L, X_U, epochs,mc_prob, mc_repeats, train_df, unobs_df):
    
 
    val_train_split = 1
    
    train_list = X_L[val_train_split:]
    val_list = X_L[:val_train_split]
    test_list = X_U 
    
    model = gf.GraphSAGE_mc(dim_h, dim_in, 12, mc_prob=mc_prob)
    # model = DataParallel(model)
    model.to(device)

    # print('Constructing training data.')
    train_data = DataLoader(train_list, batch_size=6, shuffle=True)

    # print('Constructing validation data.')	
    val_data = DataLoader(val_list, batch_size=6, shuffle=True)


    print('Training model...')
    start_time = time.time()
    train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    train_df[len(train_df.columns)] = [train_acc, train_dice, train_iou, train_loss.cpu().numpy()] 

 
    print(f'Total time taken for training: {time.time()-start_time:.4f} seconds')

    # print('Constructing test data.')
    # test_data = DataLoader(test_list)
    test_data = DataLoader(test_list, batch_size=1, shuffle=False)

    #Repeat test runs
    print('Running test graphs.')
    acc_sum = 0
    dice_sum = 0
    iou_sum = 0
    loss_sum = 0
    predict_probs_list = []
    start_time2 = time.time()

    for j in range(mc_repeats):
        loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
        loss_sum += loss.cpu().numpy()  
        acc_sum += acc
        iou_sum += iou
        dice_sum += dice
        predict_probs_list.append(predict_probs.cpu())
 

    unobs_df[len(unobs_df.columns)] = [acc_sum//mc_repeats, dice_sum//mc_repeats, iou_sum//mc_repeats, loss_sum//mc_repeats]
    print(f'Loss: {loss_sum//mc_repeats}, Acc: {acc_sum//mc_repeats}, IOU: {iou_sum//mc_repeats}, DICE: {dice_sum//mc_repeats}')    
    print(f'Total time taken for perform MC predictions: {time.time()-start_time2:.4f} seconds')

    # print(f'Testing loss: {loss:.2f}')
    # print(f'Testing accuracy: {acc:.2f}')
    # print(f'Testing iou: {iou:.2f}')
    # print(f'Testing dice: {dice:.2f}')
    # print('-'*40)

    #produces mean value array of (# nodes, # classes)
    predict_probs_arr = np.mean(np.stack(predict_probs_list, axis=0), axis=0)
    # print('predict_probs_arr.shape ', predict_probs_arr.shape)
    
    #get best patient
    predict_probs_arr = torch.from_numpy(predict_probs_arr).to(device)
    istar = Entropy(predict_probs_arr)
    # istar = Entropy(predict_probs_arr, 10)

    return istar, model, predict_probs_arr, train_df, unobs_df

In [14]:
def Entropy(probs_a):
    '''
    Calculate best instance to select using entropy. 
    Paramters:
        probs_a: (u, c) numpy array
            u is the number of unlabeled points and c is the number of classes
    Returns:
        best_patient: int
            Index of best points in the unlabeled dataset
    
    '''
    
    # probs_a dimensions: number of unlabeled poses, number of nodes, number of classes
    probs_a = probs_a.cpu().numpy()
    print(probs_a.shape)
    entropy = np.sum(probs_a*np.log(probs_a), axis=-1)
    entropy = -np.sum(entropy, axis=-1)
    print(entropy)
    best_patient = np.argmax(entropy)
#     best_patient = best_choice   
    
    return best_patient

# Passive Learning

In [15]:
def passive_learning(dim_h, dim_in, device, X_L, X_U, epochs, train_df, unobs_df):

    val_train_split = len(X_L)//3
    
    train_list = X_L[val_train_split:]
    val_list = X_L[:val_train_split]
    test_list = X_U     
    
    model = gf.GraphSAGE(dim_h, dim_in, 12)
#     model = DataParallel(model)
    model.to(device)

    # print('Constructing training data.')
    train_data = DataLoader(train_list, batch_size=6, shuffle=True)

    # print('Constructing validation data.')	
    val_data = DataLoader(val_list, batch_size=1, shuffle=True)


    print('Training model...')
    start_time = time.time()
    train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    train_df[len(unobs_df.columns)] = [train_acc, train_dice, train_iou, train_loss.cpu().numpy()] 

    print(f'Total time taken for training: {time.time()-start_time:.4f} seconds')

    # print('Constructing test data.')
    # test_data = DataLoader(test_list)
    test_data = DataLoader(test_list, batch_size=1, shuffle=False)

    #Repeat test runs
    print('Running test graphs.')
    loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
    unobs_df[len(unobs_df.columns)] = [acc, dice, iou, loss.cpu().numpy()]
    
    print(f'Loss: {loss}, Acc: {acc}, IOU: {iou}, DICE: {dice}')
    print(f'Total time taken for perform passive predictions: {time.time()-start_time:.4f} seconds')

    # print(f'Testing loss: {loss:.2f}')
    # print(f'Testing accuracy: {acc:.2f}')
    # print(f'Testing iou: {iou:.2f}')
    # print(f'Testing dice: {dice:.2f}')
    # print('-'*40)

    #get best patient
    istar = 0

    return istar, model, predict_probs.cpu().numpy(), train_df, unobs_df

# Uncertainty sampling

In [16]:
def uncertainty_sampling(dim_h, dim_in, device, X_L, X_U, epochs, train_df, unobs_df):

    val_train_split = 1
    
    train_list = X_L[val_train_split:]
    val_list = X_L[:val_train_split]
    test_list = X_U       
    
    model = gf.GraphSAGE(dim_h, dim_in, 12)
#     model = DataParallel(model)
    model.to(device)

    # print('Constructing training data.')
    train_data = DataLoader(train_list, batch_size=6, shuffle=True)

    # print('Constructing validation data.')	
    val_data = DataLoader(val_list, batch_size=1, shuffle=True)


    print('Training model...')
    start_time = time.time()
    train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    train_df[len(unobs_df.columns)] = [train_acc, train_dice, train_iou, train_loss.cpu().numpy()]    
    print(f'Total time taken for UNS training: {time.time()-start_time:.4f} seconds')

    # print('Constructing test data.')
    test_data = DataLoader(test_list, batch_size=1, shuffle=False)

    #Repeat test runs
    print('Running test graphs.')
    loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
    unobs_df[len(unobs_df.columns)] = [acc, dice, iou, loss.cpu().numpy()]

    print(f'Loss: {loss}, Acc: {acc}, IOU: {iou}, DICE: {dice}')
    print(f'Total time taken for perform UNS predictions: {time.time()-start_time:.4f} seconds')

    # print(f'Testing loss: {loss:.2f}')
    # print(f'Testing accuracy: {acc:.2f}')
    # print(f'Testing iou: {iou:.2f}')
    # print(f'Testing dice: {dice:.2f}')
    # print('-'*40)

    #get best patient
    istar = Entropy(predict_probs.cpu())

    return istar, model, predict_probs, train_df, unobs_df

# Density based sampling

In [17]:
import heapq as hp
import networkx as nx

def get_dbs(X_U, k=5):
    # print('Constructing kNN graph')

    # G=nx.Graph()

    num_nodes = X_U[0].num_nodes
    feature_arr = np.zeros((len(X_U), num_nodes, 3))
    for n in range(len(X_U)):
        feature_arr[n] = X_U[n].x.numpy()
    min_dist = np.Inf
    remove_node = -1
    for n in range(len(feature_arr)):
		
        query_point = feature_arr[n]
		
        dist_arr = np.sum((feature_arr-query_point)**2)

        if dist_arr < min_dist:
            min_dist = dist_arr
            remove_node = n

    return remove_node   


def dbs_sampling(dim_h, dim_in, device, X_L, X_U, epochs, train_df, unobs_df):
    val_train_split = 1
    
    train_list = X_L[val_train_split:]
    val_list = X_L[:val_train_split]
    test_list = X_U       
    
    model = gf.GraphSAGE(dim_h, dim_in, 12)
#     model = DataParallel(model)
    model.to(device)

    # print('Constructing training data.')
    train_data = DataLoader(train_list, batch_size=6, shuffle=True)

    # print('Constructing validation data.')	
    val_data = DataLoader(val_list, batch_size=1, shuffle=True)


    print('Training model...')
    start_time = time.time()
    train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    train_df[len(unobs_df.columns)] = [train_acc, train_dice, train_iou, train_loss.cpu().numpy()]    
    print(f'Total time taken for DBS training: {time.time()-start_time:.4f} seconds')

    # print('Constructing test data.')
    test_data = DataLoader(test_list, batch_size=1, shuffle=False)

    #Repeat test runs
    print('Running test graphs.')
    loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
    unobs_df[len(unobs_df.columns)] = [acc, dice, iou, loss.cpu().numpy()]
    
    print(f'Loss: {loss}, Acc: {acc}, IOU: {iou}, DICE: {dice}')
    print(f'Total time taken for perform DBS predictions: {time.time()-start_time:.4f} seconds')

    # print(f'Testing loss: {loss:.2f}')
    # print(f'Testing accuracy: {acc:.2f}')
    # print(f'Testing iou: {iou:.2f}')
    # print(f'Testing dice: {dice:.2f}')
    # print('-'*40)

    #get best patient
    istar = get_dbs(X_U)

    return istar, model, predict_probs, train_df, unobs_df    

# Base Run

In [18]:
def base_run(X, model_dir, load_model:bool=False):
   
    train_df = pd.DataFrame({'metrics': ['accs', 'dices', 'ious', 'losses']})
    train_df.set_index('metrics', inplace=True)
    
    unobs_df = pd.DataFrame({'metrics': ['accs', 'dices', 'ious', 'losses']})       
    unobs_df.set_index('metrics', inplace=True)  
    
    X_L, X_U = X[:start_n], X[start_n:]
    print('X_L,X_U',len(X_L), len(X_U))
    print(len(X_L[0]), len(X_U[0]))

    model_fname = f'/al_base'
    model_path = model_dir+model_fname

    model = gf.GraphSAGE(dim_h, dim_in, 12)
#     model = DataParallel(model)
    model.to(device)

    print(f'Training/Testing on Initial sampling..........')
    
    # Train/test first samples
    #split data in test, train and validation
    val_train_split = len(X_L)//3
    
    train_list = X_L[val_train_split:]
    val_list = X_L[:val_train_split]
    test_list = X_U 

    # print('Constructing training data.')
    train_data = DataLoader(train_list, batch_size=6, shuffle=True)
    val_data = DataLoader(val_list, batch_size=1, shuffle=True)    
    
    # print('Constructing test data.')    
    test_data = DataLoader(test_list, batch_size=1, shuffle=False)
    
    if load_model == False:
        print('Training model...')
        start_time = time.time()
        train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
        train_df[0] = [train_acc, train_dice.numpy(), train_iou.numpy(), train_loss.cpu().numpy()]
        train_df.to_csv(os.path.join(model_dir, 'base_training_metrics.csv'))
        print(f'Total time taken for training: {time.time()-start_time:.4f} seconds')   
        print('Saving model...')
        gf.save_model(model, model_path)
    
    else:
        print('Loading model')
        model.load_state_dict(torch.load(model_path))
        train_df = pd.read_csv(os.path.join(model_dir, 'base_training_metrics.csv'))
        train_df.set_index('metrics', inplace=True)
        pd.to_numeric(train_df['0'])
        print(train_df)

    
    print('Running test graphs.')
    loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
    unobs_df[0] = [acc, dice, iou, loss.cpu().numpy()] 
    
    # print('passive predict_probs type',type(predict_probs))
    print(predict_probs.shape)
    print(f'Testing loss: {loss:.2f}')
    print(f'Testing accuracy: {acc:.2f}')
    print(f'Testing iou: {iou:.2f}')
    print(f'Testing dice: {dice:.2f}')
    print('-'*40)
    
    f = open(os.path.join(pred_dir,f'al_base.pckl'),'wb')
    pickle.dump(predict_probs, f)
    f.close()

    # if run_i == 0:
    #     if querySelection is not 'passive':
    #         imgobj.create_images_single(test_list, predict_probs, num_instances, 'initial', querySelection.upper())
    #         imgobj.create_images_single(test_list, predict_probs, num_instances, 'initial', 'Passive')

    return X_L, X_U, train_df, unobs_df

# Load data

Load MPI Faust meshes stored as .json dictionaries representing torch_geometric Data objects

In [ ]:
gt_path = "segmentations.npz"
data = np.load(gt_path)
node_labels = data['segmentation_labels'].reshape(-1,)

print(np.unique(node_labels, return_counts=True))

data_path = "./data_dicts"
data_light_path = "./data_dicts_light"

########## Patients ###################
new_node_labels = np.load(os.path.join(data_light_path, 'node_labels.npy'))
graph_list = []       
graph_files = os.listdir(data_light_path)
graph_files.remove('node_labels.npy')
for i, graph in enumerate(graph_files):
    if '.json' in graph:
        light_g = gf.construct_graph(data_light_path, i, new_node_labels) 
        graph_list.append(light_g)



# Main

Main simulation has 5 rounds.

In [26]:
num_poses=1
num_runs = 1
# n = X.shape[0]
start_n = 20
end_n = 50
epochs = 25
dropout = 0.3
mc_repeats = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

num_instances = end_n - start_n

train_stats = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats = np.zeros((num_runs, 4, num_instances + 1))

train_stats_uns = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats_uns = np.zeros((num_runs, 4, num_instances + 1))

train_stats_mc = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats_mc = np.zeros((num_runs, 4, num_instances + 1))

train_stats_dbs = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats_dbs = np.zeros((num_runs, 4, num_instances + 1))

train_stats_qbc = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats_qbc = np.zeros((num_runs, 4, num_instances + 1))

stats_dict = {'passive':train_stats,
          'mc':train_stats_mc,
          'uns':train_stats_uns, 
          'dbs':train_stats_dbs,
          'qbc':train_stats_qbc,}

unobs_stats_dict = {'passive':unobs_stats,
          'mc':unobs_stats_mc,
          'uns': unobs_stats_uns,
          'dbs': unobs_stats_dbs,
          'qbc': unobs_stats_qbc}
# train_accs = np.zeros((num_runs, num_instances + 1))
# unobs_accs = np.zeros((num_runs, num_instances + 1))

# train_accs_qbc = np.zeros((num_runs, num_instances + 1))
# unobs_accs_qbc = np.zeros((num_runs, num_instances + 1))

# train_accs_mc = np.zeros((num_runs, num_instances + 1))
# unobs_accs_mc = np.zeros((num_runs, num_instances + 1))


# Functions for AWS run
# old_stdout = sys.stdout
# sys.stdout = open(f"./logs/passive/passive_log_{start_n}_{end_n}.log", 'w')
# sys.stdout = open(f"./logs/qbc/qbc_log_{start_n}_{end_n}.log", 'w')
# sys.stdout = open(f"./logs/mc/mc_log_{start_n}_{end_n}.log", 'w')


X = graph_list
begin_time = time.time()

for run_i in range(num_runs):
#     sys.stdout.flush()

    np.random.seed(my_seed + run_i)
    torch.manual_seed(my_seed + run_i)
    torch.cuda.manual_seed(my_seed + run_i)
    torch.backends.cudnn.deterministic = True

    print()
    print('^'*40)
    print(f'Entering active learning Run {run_i+1}..........')

    rand_i = np.arange(len(X))
    np.random.shuffle(rand_i)
    X = [X[get_i] for get_i in rand_i]

    # dim_in = graph.num_features		
    # dim_h = graph.num_nodes
    dim_in = light_g.num_features		
    dim_h = light_g.num_nodes

    gif_dir = './save_gif'
    if not os.path.exists(gif_dir):
        os.makedirs(gif_dir)

    #initialize image object
    imgobj = vf.ImageObject(gif_dir, rand_i[start_n:])
    
    for learning in ['qbc']:

        model_dir = f'./models/models_al_{learning}_debug2/al_run{run_i+1}'
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        pred_dir = f'./preds/preds_al_{learning}_debug3/al_run{run_i+1}'
        if not os.path.exists(pred_dir):
            os.makedirs(pred_dir)
        
        train_pas, unobs_pas = NewOnlineLearning(dim_h, dim_in, model_dir, pred_dir, start_n, end_n, 
                                             num_instances, learning, X, epochs, imgobj,
                                            mc_prob=dropout,
                                            mc_repeats=mc_repeats,
                                            load_model=True)
        # print(train_pas, unobs_pas)
        # # print(train_accs_j.shape)
    
        stats_dict[learning][run_i] = train_pas
        unobs_stats_dict[learning][run_i] = unobs_pas
    
        print("\n$$$$")
        print(f"{learning} Run {run_i+1} train, unobs stats")
        # print(train_pas)
        # print(unobs_pas)
        print("$$$$\n")

#     sys.stdout.flush()

        np.save(f'stats/{learning}_{start_n}_{end_n}_train_stats_run0.npy',stats_dict[learning])
        np.save(f'stats/{learning}_{start_n}_{end_n}_unobs_stats_run0.npy',stats_dict[learning])
    
print(f'Total time: {(time.time()-begin_time)/60:.4f} mins')

# sys.stdout.close()
# sys.stdout = old_stdout



cuda

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Entering active learning Run 1..........
============================== Starting QBC ==============================
X_L,X_U 20 80
4 4
Training/Testing on Initial sampling..........
Loading model
                0
metrics          
accs     0.553335
dices    0.309036
ious     0.243107
losses   2.078152
Running test graphs.
torch.Size([80, 3378, 12])
Testing loss: 2.08
Testing accuracy: 0.54
Testing iou: 0.25
Testing dice: 0.31
----------------------------------------

>>> Querying instance 1 with QBC..........

>>>>> Entering committe member 1...
Training model cmt 0...
Epoch   0 | Train Loss: 2.40 | Train Acc: 27.65% | Val Loss: 2.29 | Val Acc: 39.05%| Val IoU: 14.73%| Val DICE: 20.13%| Time: 0
Epoch   5 | Train Loss: 2.12 | Train Acc: 53.42% | Val Loss: 2.06 | Val Acc: 56.99%| Val IoU: 25.96%| Val DICE: 31.56%| Time: 2
Epoch  10 | Train Loss: 2.07 | Train Acc: 57.00% | Val Loss: 2.03 | Val Acc: 59.24%| Val IoU: 27.20%| Val DICE: 33.44%| T

In [67]:
file = './models/models_al_passive_debug2/al_run1log.txt'
ind_list = []
with open(file, 'r') as f:
    for g in f.readlines():
        i = g.strip()
        ind_list.append(int(i))

np.random.seed(830 + run_i)
torch.manual_seed(my_seed + run_i)
torch.cuda.manual_seed(my_seed + run_i)
torch.backends.cudnn.deterministic = True

random_start = np.arange(100)
np.random.shuffle(random_start)
random_start = list(random_start)
for i in ind_list:
    del random_start[i]

print(random_start)

[65, 83, 60, 28, 44, 87, 57, 55, 56, 29, 40, 5, 90, 32, 89, 41, 3, 51, 76]


In [ ]:
model_path = './models/models_al_passive_debug2/al_run1/al_instance50'

model = gf.GraphSAGE(dim_h, dim_in, 12)
model.to(device)
model.load_state_dict(torch.load(model_path))
test_list = []
for i, graph in enumerate(random_start):
        light_g = gf.construct_graph(data_light_path, i, new_node_labels) 
        test_list.append(light_g)

test_data = DataLoader(test_list, batch_size=1, shuffle=False)
loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
predict_probs = np.argmax(predict_probs.cpu(), axis=-1)

cm = np.zeros((12,12))
for i in range(len(predict_probs)):
    for n in range(predict_probs.shape[1]):
        cm[predict_probs[i, n], new_node_labels[n]] += 1

cm = np.divide(cm, (np.sum(cm, axis=1).reshape(-1, 1)), where=((np.sum(cm, axis=1).reshape(-1, 1))!= 0))
cm = np.nan_to_num(cm)
label_list = ['head', 'torso', 'left arm', 'left hand', 'right arm', 'right hand', 'left upper leg', 'left lower leg', 'left foot', 'right upper leg', 'right lower leg', 'righ foot']
plt.imshow(cm)
plt.ylabel('Predicted label')
plt.xlabel('True label')
plt.title('Confusion matrix of the model with passive after 50 instances')
plt.xticks(range(12), label_list, rotation=90)
plt.yticks(range(12), label_list)
plt.savefig('./models/models_al_uns_debug2/al_run1/cm.jpg', bbox_inches='tight')
plt.show()


# Code for plotting line graphs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

title_list = ['Accuracy', 'DICE', 'IOU', 'Loss']
color_list = ['red', 'blue', 'green', 'black', 'purple']
label_list = ['DBS', 'UNS', 'Passive', 'mc', 'qbc']
arr_1 = np.load('./stats/dbs_20_50_unobs_stats_main.npy')
arr_2 = np.load('./stats/uns_20_50_unobs_stats_main.npy')
arr_3 = np.load('./stats/passive_20_50_unobs_stats_main.npy')
arr_4 = np.load('./stats/mc_20_50_unobs_stats_main.npy')
arr_5 = np.load('./stats/qbc_20_50_unobs_stats_main.npy')
arr_4 = arr_4[0:4]
print(arr_5[0])
print(arr_1.shape)
for m in range(arr_1.shape[1]):
    for i, arr in enumerate([arr_1, arr_2, arr_3, arr_4, arr_5]):
        arr_avg = np.mean(arr, axis=0)
        arr_std = np.std(arr, axis=0)
        plt.errorbar(range(20, 51), arr_avg[m], yerr=arr_std[m], color=color_list[i], label=label_list[i] )
    plt.title(title_list[m])
    plt.xlabel('Number of training instances')
    plt.ylabel(title_list[m])	
    plt.legend()
    plt.savefig(f'./plot_{title_list[m]}.jpg')
    plt.show()
